In [3]:
import os
import time
import cloudpickle
import gzip

import numpy as np
from coffea import hist, processor
from coffea.util import load, save
import coffea                        # Using coffea version 0.6.50

import sys
sys.path.append('..')
from topcoffea.modules import samples
from topcoffea.modules.fileReader import GetFiles, GetAllInfoFromFile

outname = 'plotsTopEFT'              # Output pickle file name       
nworkers = 10                        # For futures executor

In [4]:
# Path to configuration file; Note that the path in this configuration file is commented out
cfgfile=('../cfg/samples.cfg')

In [5]:
# Temporary root file for testing
path = "root://cms-xrd-global.cern.ch//store/user/jrgonzal/nanoAODttH/8F38C0F2-E7A7-C846-BBF5-0C040E6BA839.root"

In [6]:
def get_options(path, sample, options = ""):
    if not path.endswith('/'): path += '/'
    if not sample.endswith(".root"): sample += '.root'
  #doPUweight  = 'PUweight,' if IsVarInTree(path+sample, 'puWeight') else ''
  #doJECunc    = 'JECunc,'   if IsVarInTree(path+sample, 'Jet_pt_jesTotalUp') else ''
  #doIFSR      = 'doIFSR,'   if IsVarInTree(path+sample, 'nPSWeight') and GetValOfVarInTree(path+sample, 'nPSWeight') == 4 else ''
  #useJetPtNom = 'JetPtNom,' if IsVarInTree(path+sample, 'Jet_pt_nom') else ''
  #options += doPUweight + doJECunc + doIFSR + useJetPtNom + options
    if options.endswith(','): options = options[:-1]
    return options

In [9]:
# Rewritten from topcoffea/topcoffea/modules/samples.py
def get_dictionary(cfgfile, pretend=1,test=1,verbose=1,path='',sample='',xsec='xsec',year=-1,options='',treeName='Events'):
    samplefiles = {}
    fileopt = {}
    xsecdic = {}
    sampdic = {}
    f = open(cfgfile)
    lines = f.readlines()
    for l in lines:
        l = l.replace(' ', '')
        l = l.replace('\n', '')
        if l.startswith('#'): 
            continue
        if '#' in l: l = l.split('#')[0]
        if l == '': continue
        if l.endswith(':'): l = l[:-1]
        if not ':' in l:
            if l in ['path', 'verbose', 'pretend', 'test', 'options', 'xsec', 'year', 'treeName']: continue
            else: samplefiles[l]=l
        else:
            lst = l.split(':')
            key = lst[0]
            val = lst[1] if lst[1] != '' else lst[0]
            if   key == 'pretend'   : pretend   = 1
            elif key == 'verbose'   : verbose   = int(val) if val.isdigit() else 1
            elif key == 'test'      : dotest    = 1
            elif key == 'path'      :
                path = val
                if len(lst) > 2: 
                    for v in lst[2:]: path += ':'+v
            elif key == 'options'   : options   = val
            elif key == 'xsec'      : xsec      = val
            elif key == 'year'      : year      = int(val)
            elif key == 'treeName'  : treeName  = val
            else:
                fileopt[key] = options
                if len(lst) >= 3: fileopt[key] += lst[2]
                samplefiles[key] = val
    
    
    #xsecdic = loadxsecdic(xsec, verbose)
    
    for sname in samplefiles.keys():
        sampdic[sname] = {}
        #sampdic[sname]['files']      = GetFiles(path, samplefiles[sname])
        sampdic[sname]['files']      = [path]     #temporaraily hard-coding the paths with the testing file
        extraOption = get_options(path, sampdic[sname]['files'][0].split('/')[-1])
        sampdic[sname]['options']    = fileopt[sname] + ', ' + extraOption
        sampdic[sname]['xsec']       = xsecdic[sname] if sname in xsecdic.keys() else 1
        sampdic[sname]['year']       = year
        sampdic[sname]['treeName']   = treeName
        nEvents, nGenEvents, nSumOfWeights, isData = GetAllInfoFromFile(sampdic[sname]['files'], sampdic[sname]['treeName'])
        sampdic[sname]['nEvents']       = nEvents
        sampdic[sname]['nGenEvents']    = nGenEvents
        sampdic[sname]['nSumOfWeights'] = nSumOfWeights
        sampdic[sname]['isData']        = isData        
    return(sampdic)

#   Re-assign arguments...
#   aarg = sys.argv
#   if '--pretend' in aarg or '-p' in aarg : pretend     = args.pretend
#   if '--test'    in aarg or '-t' in aarg : dotest      = args.test
#   if args.path       != ''       : path        = args.path
#   if args.options    != ''       : options     = args.options
#   if args.xsec       != 'xsec'   : xsec        = args.xsec
#   if args.year       != -1       : year        = args.year
#   if args.treeName   != 'Events' : treeName    = args.treeName
#   if args.verbose    != 0        : verbose     = int(args.verbose)
#   xsecdic = loadxsecdic(xsec, verbose)


In [10]:
samplesdict = get_dictionary(cfgfile,path=path)

In [11]:
flist = {}; xsec = {}; sow = {}; isData = {}
for k in samplesdict.keys():
    flist[k] = samplesdict[k]['files']
    xsec[k]  = samplesdict[k]['xsec']
    sow[k]   = samplesdict[k]['nSumOfWeights']
    isData[k]= samplesdict[k]['isData']

In [12]:
# Rewritten from topcoffea/analysis/topEFT/
import cloudpickle
import json
import pprint
import numpy as np
import awkward
np.seterr(divide='ignore', invalid='ignore', over='ignore')
from coffea.arrays import Initialize
#from optparse import OptionParser

from topcoffea.modules.objects import *
from topcoffea.modules.corrections import *
from topcoffea.modules.selection import *
from topcoffea.modules.HistEFT import HistEFT

# In the future these names will be read from the nanoAOD files
WCNames = ['ctW', 'ctp', 'cpQM', 'ctli', 'cQei', 'ctZ', 'cQlMi', 'cQl3i', 'ctG', 'ctlTi', 'cbW', 'cpQ3', 'ctei', 'cpt', 'ctlSi', 'cptb']


class AnalysisProcessor(processor.ProcessorABC):
    def __init__(self, samples):
        self._samples = samples

        # Create the histograms
        # In general, histograms depend on 'sample', 'channel' (final state) and 'cut' (level of selection)
        self._accumulator = processor.dict_accumulator({
        'SumOfEFTweights'  : HistEFT("SumOfWeights", WCNames, hist.Cat("sample", "sample"), hist.Bin("SumOfEFTweights", "sow", 1, 0, 2)),
        'dummy'   : hist.Hist("Dummy" , hist.Cat("sample", "sample"), hist.Bin("dummy", "Number of events", 1, 0, 1)),
        'counts'  : hist.Hist("Events", hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("counts", "Counts", 1, 0, 2)),
        'invmass' : hist.Hist("Events", hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut","cut"), hist.Bin("invmass", "$m_{\ell\ell}$ (GeV) ", 20, 0, 200)),
        'njets'   : HistEFT("Events", WCNames, hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("njets",  "Jet multiplicitu ", 10, 0, 10)),
        'nbtags'  : HistEFT("Events", WCNames, hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("nbtags", "btag multiplicitu ", 5, 0, 5)),
        'met'     : HistEFT("Events", WCNames, hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("met",    "MET (GeV)", 40, 0, 400)),
        'm3l'     : HistEFT("Events", WCNames, hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("m3l",    "$m_{3\ell}$ (GeV) ", 20, 0, 200)),
        'wleppt'  : HistEFT("Events", WCNames, hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("wleppt", "$p_{T}^{lepW}$ (GeV) ", 20, 0, 200)),
        'e0pt'    : HistEFT("Events", WCNames, hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("e0pt",   "Leading elec $p_{T}$ (GeV)", 30, 0, 300)),
        'm0pt'    : HistEFT("Events", WCNames, hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("m0pt",   "Leading muon $p_{T}$ (GeV)", 30, 0, 300)),
        'j0pt'    : HistEFT("Events", WCNames, hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("j0pt",   "Leading jet  $p_{T}$ (GeV)", 20, 0, 400)),
        'e0eta'   : HistEFT("Events", WCNames, hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("e0eta",  "Leading elec $\eta$", 20, -2.5, 2.5)),
        'm0eta'   : HistEFT("Events", WCNames, hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("m0eta",  "Leading muon $\eta$", 20, -2.5, 2.5)),
        'j0eta'   : HistEFT("Events", WCNames, hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("j0eta",  "Leading jet  $\eta$", 20, -2.5, 2.5)),
        'ht'      : HistEFT("Events", WCNames, hist.Cat("sample", "sample"), hist.Cat("channel", "channel"), hist.Cat("cut", "cut"), hist.Bin("ht",     "H$_{T}$ (GeV)", 40, 0, 800)),
        })

    @property
    def accumulator(self):
        return self._accumulator

    @property
    def columns(self):
        return self._columns

    # Main function: run on a given dataset
    def process(self, events):
        # Dataset parameters
        dataset = events.metadata['dataset']
        year   = self._samples[dataset]['year']
        xsec   = self._samples[dataset]['xsec']
        sow    = self._samples[dataset]['nSumOfWeights' ]
        isData = self._samples[dataset]['isData']
        datasets = ['SingleMuon', 'SingleElectron', 'EGamma', 'MuonEG', 'DoubleMuon', 'DoubleElectron']
        for d in datasets: 
          if d in dataset: dataset = dataset.split('_')[0] 

        # Initialize objects
        met = events.MET
        e   = events.Electron
        mu  = events.Muon
        j   = events.Jet
 

        # Electron selection
        #e['isGood'] = e.pt.zeros_like()
        e['isGood'] = isElecMVA(e.pt, e.eta, e.dxy, e.dz, e.miniPFRelIso_all, e.sip3d, e.mvaTTH, e.mvaFall17V2Iso, e.lostHits, e.convVeto, e.tightCharge, minpt=10)
        leading_e = e[e.pt.argmax()]
        leading_e = leading_e[leading_e.isGood.astype(np.bool)]

        # Muon selection
        mu['isGood'] = isMuonMVA(mu.pt, mu.eta, mu.dxy, mu.dz, mu.miniPFRelIso_all, mu.sip3d, mu.mvaTTH, mu.mediumPromptId, mu.tightCharge, minpt=10)
        leading_mu = mu[mu.pt.argmax()]
        leading_mu = leading_mu[leading_mu.isGood.astype(np.bool)]
        
        e  =  e[e .isGood.astype(np.bool)]
        mu = mu[mu.isGood.astype(np.bool)]
        nElec = e .counts
        nMuon = mu.counts

        twoLeps   = (nElec+nMuon) == 2
        threeLeps = (nElec+nMuon) == 3
        twoElec   = (nElec == 2)
        twoMuon   = (nMuon == 2)
        e0 = e[e.pt.argmax()]
        m0 = mu[mu.pt.argmax()]

        # Jet selection
        j['isgood']  = isGoodJet(j.pt, j.eta, j.jetId)
        j['isclean'] = isClean(j, e, mu)
        goodJets = j[(j.isclean)&(j.isgood)]
        njets = goodJets.counts
        ht = goodJets.pt.sum()
        j0 = goodJets[goodJets.pt.argmax()]
        nbtags = goodJets[goodJets.btagDeepFlavB > 0.2770].counts


        ##################################################################
        ### 2 same-sign leptons
        ##################################################################

        # emu
        singe = e [(nElec==1)&(nMuon==1)&(e .pt>-1)]
        singm = mu[(nElec==1)&(nMuon==1)&(mu.pt>-1)]
        em = singe.cross(singm)
        emSSmask = (em.i0.charge*em.i1.charge>0)
        emSS = em[emSSmask]
        nemSS = len(emSS.flatten())

        # ee and mumu
        # pt>-1 to preserve jagged dimensions
        ee = e [(nElec==2)&(nMuon==0)&(e.pt>-1)]
        mm = mu[(nElec==0)&(nMuon==2)&(mu.pt>-1)]

        eepairs = ee.distincts()
        eeSSmask = (eepairs.i0.charge*eepairs.i1.charge>0)
        eeonZmask  = (np.abs((eepairs.i0+eepairs.i1).mass-91)<15)
        eeoffZmask = (eeonZmask==0)

        mmpairs = mm.distincts()
        mmSSmask = (mmpairs.i0.charge*mmpairs.i1.charge>0)
        mmonZmask  = (np.abs((mmpairs.i0+mmpairs.i1).mass-91)<15)
        mmoffZmask = (mmonZmask==0)

        eeSSonZ  = eepairs[eeSSmask &  eeonZmask]
        eeSSoffZ = eepairs[eeSSmask & eeoffZmask]
        mmSSonZ  = mmpairs[mmSSmask &  mmonZmask]
        mmSSoffZ = mmpairs[mmSSmask & mmoffZmask]
        neeSS = len(eeSSonZ.flatten()) + len(eeSSoffZ.flatten())
        nmmSS = len(mmSSonZ.flatten()) + len(mmSSoffZ.flatten())

        #print('Same-sign events [ee, emu, mumu] = [%i, %i, %i]'%(neeSS, nemSS, nmmSS))

        # Cuts
        eeSSmask   = (eeSSmask[eeSSmask].counts>0)
        mmSSmask   = (mmSSmask[mmSSmask].counts>0)
        eeonZmask  = (eeonZmask[eeonZmask].counts>0)
        eeoffZmask = (eeoffZmask[eeoffZmask].counts>0)
        mmonZmask  = (mmonZmask[mmonZmask].counts>0)
        mmoffZmask = (mmoffZmask[mmoffZmask].counts>0)
        emSSmask    = (emSSmask[emSSmask].counts>0)

        # njets

        ##################################################################
        ### 3 leptons
        ##################################################################

        # eem
        muon_eem = mu[(nElec==2)&(nMuon==1)&(mu.pt>-1)]
        elec_eem =  e[(nElec==2)&(nMuon==1)&( e.pt>-1)]
        ee_eem   = elec_eem.distincts()
        ee_eemZmask     = (ee_eem.i0.charge*ee_eem.i1.charge<1)&(np.abs((ee_eem.i0+ee_eem.i1).mass-91)<15)
        ee_eemOffZmask  = (ee_eem.i0.charge*ee_eem.i1.charge<1)&(np.abs((ee_eem.i0+ee_eem.i1).mass-91)>15)
        ee_eemZmask     = (ee_eemZmask[ee_eemZmask].counts>0)
        ee_eemOffZmask  = (ee_eemOffZmask[ee_eemOffZmask].counts>0)

        eepair_eem     = (ee_eem.i0+ee_eem.i1)
        trilep_eem     = eepair_eem.cross(muon_eem)
        trilep_eem     = (trilep_eem.i0+trilep_eem.i1) 

        # mme
        muon_mme = mu[(nElec==1)&(nMuon==2)&(mu.pt>-1)]
        elec_mme =  e[(nElec==1)&(nMuon==2)&( e.pt>-1)]
        mm_mme   = muon_mme.distincts()
        mm_mmeZmask     = (mm_mme.i0.charge*mm_mme.i1.charge<1)&(np.abs((mm_mme.i0+mm_mme.i1).mass-91)<15)
        mm_mmeOffZmask  = (mm_mme.i0.charge*mm_mme.i1.charge<1)&(np.abs((mm_mme.i0+mm_mme.i1).mass-91)>15)
        mm_mmeZmask     = (mm_mmeZmask[mm_mmeZmask].counts>0)
        mm_mmeOffZmask  = (mm_mmeOffZmask[mm_mmeOffZmask].counts>0)

        mmpair_mme     = (mm_mme.i0+mm_mme.i1)
        trilep_mme     = mmpair_mme.cross(elec_mme)
        trilep_mme     = (trilep_mme.i0+trilep_mme.i1)
        mZ_mme  = mmpair_mme.mass
        mZ_eem  = eepair_eem.mass
        m3l_eem = trilep_eem.mass
        m3l_mme = trilep_mme.mass


        ### eee and mmm
        eee =   e[(nElec==3)&(nMuon==0)&( e.pt>-1)] 
        mmm =  mu[(nElec==0)&(nMuon==3)&(mu.pt>-1)] 
        # Create pairs
        ee_pairs = eee.argchoose(2)
        mm_pairs = mmm.argchoose(2)

        # Select pairs that are SFOS.
        eeSFOS_pairs = ee_pairs[(np.abs(eee[ee_pairs.i0].pdgId) == np.abs(eee[ee_pairs.i1].pdgId)) & (eee[ee_pairs.i0].charge != eee[ee_pairs.i1].charge)]
        mmSFOS_pairs = mm_pairs[(np.abs(mmm[mm_pairs.i0].pdgId) == np.abs(mmm[mm_pairs.i1].pdgId)) & (mmm[mm_pairs.i0].charge != mmm[mm_pairs.i1].charge)]
        # Find the pair with mass closest to Z.
        eeOSSFmask = eeSFOS_pairs[np.abs((eee[eeSFOS_pairs.i0] + eee[eeSFOS_pairs.i1]).mass - 91.2).argmin()]
        onZmask_ee = np.abs((eee[eeOSSFmask.i0] + eee[eeOSSFmask.i1]).mass - 91.2) < 15
        mmOSSFmask = mmSFOS_pairs[np.abs((mmm[mmSFOS_pairs.i0] + mmm[mmSFOS_pairs.i1]).mass - 91.2).argmin()]
        onZmask_mm = np.abs((mmm[mmOSSFmask.i0] + mmm[mmOSSFmask.i1]).mass - 91.2) < 15
        offZmask_ee = np.abs((eee[eeOSSFmask.i0] + eee[eeOSSFmask.i1]).mass - 91.2) > 15
        offZmask_mm = np.abs((mmm[mmOSSFmask.i0] + mmm[mmOSSFmask.i1]).mass - 91.2) > 15

        # Create masks
        eeeOnZmask  = onZmask_ee[onZmask_ee].counts>0
        eeeOffZmask = offZmask_ee[offZmask_ee].counts>0
        mmmOnZmask  = onZmask_mm[onZmask_mm].counts>0
        mmmOffZmask = offZmask_mm[offZmask_mm].counts>0
    
        # Leptons from Z
        eZ0= eee[eeOSSFmask.i0]
        eZ1= eee[eeOSSFmask.i1]
        mZ0= mmm[mmOSSFmask.i0]
        mZ1= mmm[mmOSSFmask.i1]

        # Leptons from W
        eW = eee[~eeOSSFmask.i0 | ~eeOSSFmask.i1]
        mW = mmm[~mmOSSFmask.i0 | ~mmOSSFmask.i1]

        eZ = eee[eeOSSFmask.i0] + eee[eeOSSFmask.i1]
        triElec = eZ + eW
        mZ = mmm[mmOSSFmask.i0] + mmm[mmOSSFmask.i1]
        triMuon = mZ + mW

        mZ_eee  = eZ.mass
        m3l_eee = triElec.mass
        mZ_mmm  = mZ.mass
        m3l_mmm = triMuon.mass
    
        # Triggers
        #passTrigger = lambda events, n, m, o : np.ones_like(events['MET_pt'], dtype=np.bool) # XXX
        trig_eeSS = passTrigger(events,'ee',isData,dataset)
        trig_mmSS = passTrigger(events,'mm',isData,dataset)
        trig_emSS = passTrigger(events,'em',isData,dataset)
        trig_eee  = passTrigger(events,'eee',isData,dataset)
        trig_mmm  = passTrigger(events,'mmm',isData,dataset)
        trig_eem  = passTrigger(events,'eem',isData,dataset)
        trig_mme  = passTrigger(events,'mme',isData,dataset)


        # MET filters

        # Weights
        genw = np.ones_like(events['MET_pt']) if isData else events['genWeight']
        weights = processor.Weights(events.size)
        weights.add('norm',genw if isData else (xsec/sow)*genw)
        eftweights = events['EFTfitCoefficients'] if hasattr(events, "EFTfitCoefficients") else []

        # Selections and cuts
        selections = processor.PackedSelection()
        channels2LSS = ['eeSSonZ', 'eeSSoffZ', 'mmSSonZ', 'mmSSoffZ', 'emSS']
        selections.add('eeSSonZ',  (eeonZmask)&(eeSSmask)&(trig_eeSS))
        selections.add('eeSSoffZ', (eeoffZmask)&(eeSSmask)&(trig_eeSS))
        selections.add('mmSSonZ',  (mmonZmask)&(mmSSmask)&(trig_mmSS))
        selections.add('mmSSoffZ', (mmoffZmask)&(mmSSmask)&(trig_mmSS))
        selections.add('emSS',     (emSSmask)&(trig_emSS))

        channels3L = ['eemSSonZ', 'eemSSoffZ', 'mmeSSonZ', 'mmeSSoffZ']
        selections.add('eemSSonZ',   (ee_eemZmask)&(trig_eem))
        selections.add('eemSSoffZ',  (ee_eemOffZmask)&(trig_eem))
        selections.add('mmeSSonZ',   (mm_mmeZmask)&(trig_mme))
        selections.add('mmeSSoffZ',  (mm_mmeOffZmask)&(trig_mme))

        channels3L += ['eeeSSonZ', 'eeeSSoffZ', 'mmmSSonZ', 'mmmSSoffZ']
        selections.add('eeeSSonZ',   (eeeOnZmask)&(trig_eee))
        selections.add('eeeSSoffZ',  (eeeOffZmask)&(trig_eee))
        selections.add('mmmSSonZ',   (mmmOnZmask)&(trig_mmm))
        selections.add('mmmSSoffZ',  (mmmOffZmask)&(trig_mmm))

        levels = ['base', '2jets', '4jets', '4j1b', '4j2b']
        selections.add('base', (nElec+nMuon>=2))
        selections.add('2jets',(njets>=2))
        selections.add('4jets',(njets>=4))
        selections.add('4j1b',(njets>=4)&(nbtags>=1))
        selections.add('4j2b',(njets>=4)&(nbtags>=2))

        # Variables
        invMass_eeSSonZ  = ( eeSSonZ.i0+ eeSSonZ.i1).mass
        invMass_eeSSoffZ = (eeSSoffZ.i0+eeSSoffZ.i1).mass
        invMass_mmSSonZ  = ( mmSSonZ.i0+ mmSSonZ.i1).mass
        invMass_mmSSoffZ = (mmSSoffZ.i0+mmSSoffZ.i1).mass
        invMass_emSS     = (emSS.i0+emSS.i1).mass

        varnames = {}
        varnames['met'] = met.pt
        varnames['ht'] = ht
        varnames['njets'] = njets
        varnames['nbtags'] = nbtags
        varnames['invmass'] = {
          'eeSSonZ'   : invMass_eeSSonZ,
          'eeSSoffZ'  : invMass_eeSSoffZ,
          'mmSSonZ'   : invMass_mmSSonZ,
          'mmSSoffZ'  : invMass_mmSSoffZ,
          'emSS'      : invMass_emSS,
          'eemSSonZ'  : mZ_eem,
          'eemSSoffZ' : mZ_eem,
          'mmeSSonZ'  : mZ_mme,
          'mmeSSoffZ' : mZ_mme,
          'eeeSSonZ'  : mZ_eee,
          'eeeSSoffZ' : mZ_eee,
          'mmmSSonZ'  : mZ_mmm,
          'mmmSSoffZ' : mZ_mmm,
        }
        varnames['m3l'] = {
          'eemSSonZ'  : m3l_eem,
          'eemSSoffZ' : m3l_eem,
          'mmeSSonZ'  : m3l_mme,
          'mmeSSoffZ' : m3l_mme,
          'eeeSSonZ'  : m3l_eee,
          'eeeSSoffZ' : m3l_eee,
          'mmmSSonZ'  : m3l_mmm,
          'mmmSSoffZ' : m3l_mmm,
        }
        varnames['e0pt' ] = e0.pt
        varnames['e0eta'] = e0.eta
        varnames['m0pt' ] = m0.pt
        varnames['m0eta'] = m0.eta
        varnames['j0pt' ] = j0.pt
        varnames['j0eta'] = j0.eta
        varnames['counts'] = np.ones_like(events.MET.pt, dtype=np.int) 

        # fill Histos
        hout = self.accumulator.identity()
        allweights = weights.weight().flatten()
        #hout['dummy'].fill(sample=dataset, dummy=varnames['counts'], weight=np.ones_like(events.MET.pt, dtype=np.int))
        hout['SumOfEFTweights'].fill(eftweights, sample=dataset, SumOfEFTweights=varnames['counts'], weight=allweights)

        for var, v in varnames.items():
            for ch in channels2LSS+channels3L:
                for lev in levels:
                    weight = weights.weight()
                    cuts = [ch] + [lev]
                    cut = selections.all(*cuts)
                    weights_flat = weight[cut].flatten()
                    weights_ones = np.ones_like(weights_flat, dtype=np.int)
                    eftweightsvalues = eftweights[cut] if len(eftweights) > 0 else []
                    if var == 'invmass':
                        if   ch in ['eeeSSoffZ', 'mmmSSoffZ']: continue
                        elif ch in ['eeeSSonZ' , 'mmmSSonZ' ]: continue #values = v[ch]
                        else                                 : values = v[ch][cut].flatten()
                        hout['invmass'].fill(sample=dataset, channel=ch, cut=lev, invmass=values, weight=weights_flat)
                    elif var == 'm3l': 
                        if ch in ['eeSSonZ','eeSSoffZ', 'mmSSonZ', 'mmSSoffZ','emSS', 'eeeSSoffZ', 'mmmSSoffZ', 'eeeSSonZ' , 'mmmSSonZ']: continue
                        values = v[ch][cut].flatten()
                        hout['m3l'].fill(eftweightsvalues, sample=dataset, channel=ch, cut=lev, m3l=values, weight=weights_flat)
                    else:
                        values = v[cut].flatten()
                        if   var == 'ht'    : hout[var].fill(eftweightsvalues, ht=values, sample=dataset, channel=ch, cut=lev, weight=weights_flat)
                        elif var == 'met'   : hout[var].fill(eftweightsvalues, met=values, sample=dataset, channel=ch, cut=lev, weight=weights_flat)
                        elif var == 'njets' : hout[var].fill(eftweightsvalues, njets=values, sample=dataset, channel=ch, cut=lev, weight=weights_flat)
                        elif var == 'nbtags': hout[var].fill(eftweightsvalues, nbtags=values, sample=dataset, channel=ch, cut=lev, weight=weights_flat)
                        elif var == 'counts': hout[var].fill(counts=values, sample=dataset, channel=ch, cut=lev, weight=weights_ones)
                        elif var == 'j0eta' : 
                            if lev == 'base': continue
                            hout[var].fill(eftweightsvalues, j0eta=values, sample=dataset, channel=ch, cut=lev, weight=weights_flat)
                        elif var == 'e0pt'  : 
                            if ch in ['mmSSonZ', 'mmSSoffZ', 'mmmSSoffZ', 'mmmSSonZ']: continue
                            hout[var].fill(eftweightsvalues, e0pt=values, sample=dataset, channel=ch, cut=lev, weight=weights_flat)
                        elif var == 'm0pt'  : 
                            if ch in ['eeSSonZ', 'eeSSoffZ', 'eeeSSoffZ', 'eeeSSonZ']: continue
                            hout[var].fill(eftweightsvalues, m0pt=values, sample=dataset, channel=ch, cut=lev, weight=weights_flat)
                        elif var == 'e0eta' : 
                            if ch in ['mmSSonZ', 'mmSSoffZ', 'mmmSSoffZ', 'mmmSSonZ']: continue
                            hout[var].fill(eftweightsvalues, e0eta=values, sample=dataset, channel=ch, cut=lev, weight=weights_flat)
                        elif var == 'm0eta':
                            if ch in ['eeSSonZ', 'eeSSoffZ', 'eeeSSoffZ', 'eeeSSonZ']: continue
                            hout[var].fill(eftweightsvalues, m0eta=values, sample=dataset, channel=ch, cut=lev, weight=weights_flat)
                        elif var == 'j0pt'  : 
                            if lev == 'base': continue
                            hout[var].fill(eftweightsvalues, j0pt=values, sample=dataset, channel=ch, cut=lev, weight=weights_flat)

        return hout

    def postprocess(self, accumulator):
        return accumulator


In [14]:
processor_instance = AnalysisProcessor(samplesdict)

In [15]:
flist

{'DYJetsToLL_MLL50': ['root://xcache//store/user/jrgonzal/nanoAODttH/8F38C0F2-E7A7-C846-BBF5-0C040E6BA839.root'],
 'DYJetsToLL_M_10to50': ['root://xcache//store/user/jrgonzal/nanoAODttH/8F38C0F2-E7A7-C846-BBF5-0C040E6BA839.root'],
 'ZZTo2L2Nu': ['root://xcache//store/user/jrgonzal/nanoAODttH/8F38C0F2-E7A7-C846-BBF5-0C040E6BA839.root'],
 'ZZTo4L': ['root://xcache//store/user/jrgonzal/nanoAODttH/8F38C0F2-E7A7-C846-BBF5-0C040E6BA839.root'],
 'WZTo3LNU': ['root://xcache//store/user/jrgonzal/nanoAODttH/8F38C0F2-E7A7-C846-BBF5-0C040E6BA839.root'],
 'WWTo2L2Nu': ['root://xcache//store/user/jrgonzal/nanoAODttH/8F38C0F2-E7A7-C846-BBF5-0C040E6BA839.root'],
 'tW_noFullHad': ['root://xcache//store/user/jrgonzal/nanoAODttH/8F38C0F2-E7A7-C846-BBF5-0C040E6BA839.root'],
 'tbarW_noFullHad': ['root://xcache//store/user/jrgonzal/nanoAODttH/8F38C0F2-E7A7-C846-BBF5-0C040E6BA839.root'],
 'TT': ['root://xcache//store/user/jrgonzal/nanoAODttH/8F38C0F2-E7A7-C846-BBF5-0C040E6BA839.root'],
 'TTsemilep': ['root:/

In [10]:
# Testing with just with one item from filelist, since I don't have Xuan's files,  all paths in the filelist are copies of each other
flist = {'DYJetsToLL_MLL50': ["root://xcache//store/user/jrgonzal/nanoAODttH/8F38C0F2-E7A7-C846-BBF5-0C040E6BA839.root"]}

In [16]:
tstart = time.time()
output = processor.run_uproot_job(flist, 
                                  treename='Events', 
                                  processor_instance=processor_instance, 
                                  executor=processor.futures_executor, 
                                  executor_args={'nano':True}, 
                                  chunksize=500000)

/opt/conda/lib/python3.8/site-packages/coffea/util.py:98: Awkward0Warning: In coffea version 0.7 (target date: January 2021), this will be an error.
(Set awkward1.deprecations_as_errors = True to get a stack trace now.)
TypeError: <class 'coffea.nanoaod.nanoevents.NanoEvents'> relies exclusively on awkward 0.x and will be removed in upcoming versions of coffea!
  warnings.warn(message, Awkward0Warning)
/opt/conda/lib/python3.8/site-packages/coffea/util.py:98: Awkward0Warning: In coffea version 0.7 (target date: January 2021), this will be an error.
(Set awkward1.deprecations_as_errors = True to get a stack trace now.)
TypeError: <class 'coffea.nanoaod.nanoevents.NanoCollection'> relies exclusively on awkward 0.x and will be removed in upcoming versions of coffea!
  warnings.warn(message, Awkward0Warning)


In [17]:
dt = time.time() - tstart

nbins = sum(sum(arr.size for arr in h._sumw.values()) for h in output.values() if isinstance(h, hist.Hist))
nfilled = sum(sum(np.sum(arr > 0) for arr in h._sumw.values()) for h in output.values() if isinstance(h, hist.Hist))
print("Filled %.0f bins, nonzero bins: %1.1f %%" % (nbins, 100*nfilled/nbins,))
print("Processing time: %1.2f s with %i workers (%.2f s cpu overall)" % (dt, nworkers, dt*nworkers, ))

os.system("mkdir -p histos/")
print('Saving output in %s...'%("histos/" + outname + ".pkl.gz"))
with gzip.open("histos/" + outname + ".pkl.gz", "wb") as fout:
    cloudpickle.dump(output, fout)
print('Done!')

Filled 161460 bins, nonzero bins: 74.3 %
Processing time: 3911.25 s with 10 workers (39112.51 s cpu overall)
Saving output in histos/plotsTopEFT.pkl.gz...
Done!


In [18]:
output

{'SumOfEFTweights': <HistEFT (sample,SumOfEFTweights) instance at 0x7ffadb6b1ca0>,
 'dummy': <Hist (sample,dummy) instance at 0x7ffadbe1de80>,
 'counts': <Hist (sample,channel,cut,counts) instance at 0x7ffadbe36ca0>,
 'invmass': <Hist (sample,channel,cut,invmass) instance at 0x7ffadbfcc1c0>,
 'njets': <HistEFT (sample,channel,cut,njets) instance at 0x7ffadbfcc070>,
 'nbtags': <HistEFT (sample,channel,cut,nbtags) instance at 0x7ffadbfcc4f0>,
 'met': <HistEFT (sample,channel,cut,met) instance at 0x7ffadbfcc790>,
 'm3l': <HistEFT (sample,channel,cut,m3l) instance at 0x7ffadbe36a30>,
 'wleppt': <HistEFT (sample,channel,cut,wleppt) instance at 0x7ffadc133e80>,
 'e0pt': <HistEFT (sample,channel,cut,e0pt) instance at 0x7ffadbe369a0>,
 'm0pt': <HistEFT (sample,channel,cut,m0pt) instance at 0x7ffadbe361c0>,
 'j0pt': <HistEFT (sample,channel,cut,j0pt) instance at 0x7ffadc15e130>,
 'e0eta': <HistEFT (sample,channel,cut,e0eta) instance at 0x7ffadbe36400>,
 'm0eta': <HistEFT (sample,channel,cut,m0e